
# Project 1: Build an LLM Playground

Welcome! In this project, you’ll learn foundations of large language models (LLMs). We’ll keep the code minimal and the explanations high‑level so that anyone who can run a Python cell can follow along.  

We'll be using Google Colab for this project. Colab is a free, browser-based platform that lets you run Python code and machine learning models without installing anything on your local computer. Click the button below to open this notebook directly in Google Colab and get started!


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bytebyteai/ai-eng-projects/blob/main/project_1/lm_playground.ipynb)

---
## Learning Objectives  
* **Tokenization** and how raw text is tokenized into a sequene of discrete tokens
* Inspect **GPT2** and **Transformer architecture**
* Loading pre-trained LLMs using **Hugging Face**
* **Decoding strategies** to generate text from LLMs
* Completion versus **intrusction fine-tuned** LLMs


Let's get started!

In [ ]:
import torch, transformers, tiktoken
print("torch", torch.__version__, "| transformers", transformers.__version__)

# 1 - Tokenization

A neural network can’t digest raw text. They need **numbers**. Tokenization is the process of converting text into IDs. In this section, you'll learn how tokenization is implemented in practice.

Tokenization methods generally fall into three categories:
1. Word-level
2. Character-level
3. Subword-level

### 1.1 - Word‑level tokenization

Split text on whitespace and store each **word** as a token.

In [72]:
# 1. Tiny corpus
corpus = [
    "The quick brown fox jumps over the lazy dog",
    "Tokenization converts text to numbers",
    "Large language models predict the next token"
]

# 2. Build the vocabulary
PAD, UNK = "[PAD]", "[UNK]"
vocab = []
word2id = {}
id2word = {}

# My Code

#define vocab
for i in corpus:
  vocab.extend(i.split(' '))

for index, j in enumerate(vocab):
  word2id[j] = index
  id2word[index] = str(j)


print(f"Vocabulary size: {len(vocab)} words")
print("First 15 vocab entries:", vocab[:15])
print(word2id)
print(id2word)
print(len(word2id))

# 3. Encode / decode
# encoding handles another portion of updating the dictionary dynamically for the new entries
# do this at encoding. Do not do it at decoding
def encode(text):
  encoded = []
  for index, i in enumerate(text.split(' ')):
    if i not in word2id:
      word2id[i] = (int(len(word2id)) + 1)
      id2word[(len(word2id))] = i # use the length of the last array since it has the update value
    encoded.append(word2id[i])
  return encoded

def decode(ids):
  decoded = []
  for j in ids:
    decoded.append(id2word[j])

  return decoded

# 4. Demo
sample = "The brown unicorn sdfsd sfeo"
ids = encode(sample)
print(word2id)
print(id2word)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Decoded    :", recovered)

Vocabulary size: 21 words
First 15 vocab entries: ['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', 'Tokenization', 'converts', 'text', 'to', 'numbers', 'Large']
{'The': 0, 'quick': 1, 'brown': 2, 'fox': 3, 'jumps': 4, 'over': 5, 'the': 18, 'lazy': 7, 'dog': 8, 'Tokenization': 9, 'converts': 10, 'text': 11, 'to': 12, 'numbers': 13, 'Large': 14, 'language': 15, 'models': 16, 'predict': 17, 'next': 19, 'token': 20}
{0: 'The', 1: 'quick', 2: 'brown', 3: 'fox', 4: 'jumps', 5: 'over', 6: 'the', 7: 'lazy', 8: 'dog', 9: 'Tokenization', 10: 'converts', 11: 'text', 12: 'to', 13: 'numbers', 14: 'Large', 15: 'language', 16: 'models', 17: 'predict', 18: 'the', 19: 'next', 20: 'token'}
20
{'The': 0, 'quick': 1, 'brown': 2, 'fox': 3, 'jumps': 4, 'over': 5, 'the': 18, 'lazy': 7, 'dog': 8, 'Tokenization': 9, 'converts': 10, 'text': 11, 'to': 12, 'numbers': 13, 'Large': 14, 'language': 15, 'models': 16, 'predict': 17, 'next': 19, 'token': 20, 'unicorn': 21, 'sdfsd': 22, 'sfeo': 23

Word-level tokenization has two major limitations:
1. Large vocabulary size
2. Out-of-vocabulary (OOV) issue

### 1.2 - Character‑level tokenization

Every single character (including spaces and emojis) gets its own ID. This guarantees zero out‑of‑vocabulary issues but very long sequences.

In [4]:
# 1. Build a fixed vocabulary # a–z + A–Z + padding + unkwown
import string

vocab = []
char2id = {}
id2char = {}
"""
YOUR CODE HERE
"""
vocab_total="abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
for letter in vocab_total:
  vocab.append(letter)
vocab.append(" ")
vocab.append("?")
for index, i in enumerate(vocab):
  char2id[i] = index
  id2char[index] = i

print(f"Vocabulary size: {len(vocab)} (52 letters + 2 specials)")

# 2. Encode / decode
def encode(text):
    """
    YOUR CODE HERE
    """
    encoded = []
    for letter in text:
      if letter not in char2id:
        encoded.append(char2id["?"])
      else:
        encoded.append(char2id[letter])
    return encoded

def decode(ids):
    """
    YOUR CODE HERE
    """
    decoded =[]
    for letter in ids:
      decoded.append(id2char[letter])

    return decoded

# 3. Demo
sample = "Hello"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Decoded    :", recovered)


Vocabulary size: 54 (52 letters + 2 specials)

Input text : Hello
Token IDs  : [33, 4, 11, 11, 14]
Decoded    : ['H', 'e', 'l', 'l', 'o']


### 1.3 - Subword‑level tokenization

Sub-word methods such as `Byte-Pair Encoding (BPE)`, `WordPiece`, and `SentencePiece` **learn** the most common character and gorup them into new tokens. For example, the word `unbelievable` might turn into three tokens: `["un", "believ", "able"]`. This approach strikes a balance between word-level and character-level methods and fix their limitations.

For example, `BPE` algorithm forms the vocabulary using the following steps:
1. **Start with bytes** → every character is its own token.  
2. **Count all adjacent pairs** in a huge corpus.  
3. **Merge the most frequent pair** into a new token.  
   *Repeat steps 2-3* until you hit the target vocab size (e.g., 50 k).

Let's see `BPE` in practice.

In [68]:
# 1. Load a pretrained BPE tokenizer (GPT-2 uses BPE).
# Refer to  https://huggingface.co/docs/transformers/en/fast_tokenizers

from transformers import AutoTokenizer


bpe_tok = AutoTokenizer.from_pretrained("gpt2")



# 2. Encode / decode
def encode(text):
    """
    YOUR CODE HERE
    """
    return bpe_tok.encode(text)



def decode(ids):
    """
    YOUR CODE HERE
    """
    return bpe_tok.decode(ids)

# 3. Demo
sample = "Unbelievable tokenization powers! 🚀"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Tokens     :", bpe_tok.convert_ids_to_tokens(ids))
print("Decoded    :", recovered)



Input text : Unbelievable tokenization powers! 🚀
Token IDs  : [3118, 6667, 11203, 540, 11241, 1634, 5635, 0, 12520, 248, 222]
Tokens     : ['Un', 'bel', 'iev', 'able', 'Ġtoken', 'ization', 'Ġpowers', '!', 'ĠðŁ', 'ļ', 'Ģ']
Decoded    : Unbelievable tokenization powers! 🚀


### 1.4 - TikToken

`tiktoken` is a production-ready library which offers high‑speed tokenization used by OpenAI models.  
Let's compare the older **gpt2** encoding with the newer **cl100k_base** used in GPT‑4.

In [67]:
# Use gpt2 and cl100k_base to encode and decode the following text
# Refer to https://github.com/openai/tiktoken
import tiktoken


sentence = "The 🌟 star-player scored 40 points!"

"""
YOUR CODE HERE
"""

gpt2_base = tiktoken.get_encoding("gpt2")
gpt2_encode = gpt2_base.encode(sentence)

#enc = SimpleBytePairEncoding.from_tiktoken("cl100k_base")
cl100k_base = tiktoken.get_encoding("cl100k_base")
gpt4_encode = cl100k_base.encode(sentence)


print("\nInput text :", sentence)
print("\nGPT2 Encode :", gpt2_encode)
print("\nl100kbase Encode :", gpt4_encode)



Input text : The 🌟 star-player scored 40 points!

GPT2 Encode : [464, 12520, 234, 253, 3491, 12, 7829, 7781, 2319, 2173, 0]

l100kbase Encode : [791, 11410, 234, 253, 6917, 43467, 16957, 220, 1272, 3585, 0]


Experiment: try new sentences, emojis, code snippets, or other languages. If you are interested, try implementing the BPE algorithm yourself.

### 1.5 - Key Takeaways

* **Word‑level**: simple but brittle (OOV problems).  
* **Character‑level**: robust but produces long sequences.  
* **BPE / Byte‑Level BPE**: middle ground used by most LLMs.  
* **tiktoken**: shows how production models tokenize with pre‑trained sub‑word vocabularies.

# 2. What is a Language Model?

At its core, a **language model (LM)** is just a *very large* mathematical function built from many neural-network layers.  
Given a sequence of tokens `[t₁, t₂, …, tₙ]`, it learns to output a probability for the next token `tₙ₊₁`.


Each layer applies a simple operation (matrix multiplication, attention, etc.). Stacking hundreds of these layers lets the model capture patterns and statistical relations from text. The final output is a vector of scores that says, “how likely is each possible token to come next?”

> Think of the whole network as **one gigantic equation** whose parameters were tuned during training to minimize prediction error.



### 2.1 - A Single `Linear` Layer

Before we explore Transformer, let’s start tiny:

* A **Linear layer** performs `y = Wx + b`  
  * `x` – input vector  
  * `W` – weight matrix (learned)  
  * `b` – bias vector (learned)

Although this looks basic, chaining thousands of such linear transforms (with nonlinearities in between) gives neural nets their expressive power.


In [43]:
import torch.nn as nn
class Linear(nn.Module):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        """
        YOUR CODE HERE
        """
        self.lin = nn.Linear(in_features, out_features)

    def forward(self, x):
        """
        YOUR CODE HERE
        """
        x = self.lin(x)
        return x

In [50]:
import torch.nn as nn, torch

lin = nn.Linear(3, 2)
x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)
print("Bias   :", lin.bias)
print("Output :", lin(x))


Input : tensor([ 1.0000, -1.0000,  0.5000])
Weights: Parameter containing:
tensor([[-0.3234,  0.3007,  0.1114],
        [ 0.0468, -0.3790,  0.1659]], requires_grad=True)
Bias   : Parameter containing:
tensor([0.1543, 0.2981], requires_grad=True)
Output : tensor([-0.4141,  0.8068], grad_fn=<ViewBackward0>)


### 2.2 - A `Transformer` Layer

Most LLMs are a **stack of identical Transformer blocks**. Each block fuses two main components:

| Step | What it does | Where it lives in code |
|------|--------------|------------------------|
| **Multi-Head Self-Attention** | Every token looks at every other token and decides *what matters*. | `block.attn` |
| **Feed-Forward Network (MLP)** | Re-mixes information token-by-token. | `block.mlp` |

Below, we load the smallest public GPT-2 (124 M parameters), grab its *first* block, and inspect the pieces.


In [51]:
import torch
from transformers import GPT2LMHeadModel

# Load the 124 M-parameter GPT-2 and inspect its layers (12 layers)
"""
YOUR CODE HERE
"""
gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")
print(gpt2.config)
# comments
# takes about 10 seconds to load in
# 12 layers, which tracks with the lexture
# activation function is gelu_new


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "dtype": "float32",
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.57.0",
  "use_cache": true,
  "vocab_size": 50257
}



In [88]:
# Run a tiny forward pass through the first block
seq_len = 8
dummy_tokens = torch.randint(0, gpt2.config.vocab_size, (1, seq_len))
with torch.no_grad():
    # Embed tokens + positions the same way GPT-2 does
    # Forward through one layer
    """
    YOUR CODE HERE
    """
    print(dummy_tokens)
    out = gpt2(dummy_tokens)


print("\nOutput shape :", out.logits.shape) # (batch, seq_len, hidden_size)

tensor([[ 1517, 42729, 13277,  7734,   420, 43096, 23879,  4145]])

Output shape : torch.Size([1, 8, 50257])


### 2.3 - Inside GPT-2

GPT-2 is just many of those modules arranged in a repeating *block*. Let's print the modules inside the Transformer.

In [91]:
# Print the name and modules inside gpt2
"""
YOUR CODE HERE
"""
for name, module in gpt2.named_modules():
    print(name, module)

#unreadable honestly
# but does go through all twelve layers so you can see what's going on


 GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)
transformer GPT2Model(
  (wte): Embedding(50257, 768

As you can see, the Transformer holds various modules, arranged from a list of blocks (`h`). The following table summarizes these modules:

| Step | What it does | Why it matters |
|------|--------------|----------------|
| **Token → Embedding** | Converts IDs to vectors | Gives the model a numeric “handle” on words |
| **Positional Encoding** | Adds “where am I?” info | Order matters in language |
| **Multi-Head Self-Attention** | Each token asks “which other tokens should I look at?” | Lets the model relate words across a sentence |
| **Feed-Forward Network** | Two stacked Linear layers with a non-linearity | Mixes information and adds depth |
| **LayerNorm & Residual** | Stabilize training and help gradients flow | Keeps very deep networks trainable |


### 2.4 LLM's output

Passing a token sequence through an **LLM** yields a tensor of **logits** with shape  
`(batch_size, seq_len, vocab_size)`.  
Applying `softmax` on the last dimension turns those logits into probabilities.

The cell below feeds an 8-token dummy sequence, prints the logits shape, and shows the five most likely next tokens for the final position.


In [119]:
import torch, torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# Load gpt2 model and tokenizer
"""
YOUR CODE HERE
"""
tokenizer = GPT2TokenizerFast.from_pretrained("openai-community/gpt2")
gpt2 = GPT2LMHeadModel.from_pretrained("openai-community/gpt2")

# Tokenize input text
text = "Hello my name"
"""
YOUR CODE HERE
"""
inputs = tokenizer(text, return_tensors="pt")
print(inputs)
# {'input_ids': tensor([[15496,   616,  1438]]), 'attention_mask': tensor([[1, 1, 1]])}

# Get logits by passing the ids to the gpt2 model.
"""
YOUR CODE HERE
"""
out = gpt2(inputs.input_ids)
logits = out.logits
print(out)

print("Logits shape :", logits.shape)

# Predict next token
"""
YOUR CODE HERE
"""
last_token_logits = logits[0, -1, :] # last logit has the next token calculation because that's the feedthrough mechanism
probabilities = F.softmax(last_token_logits, dim=-1)
top5_probs, top5_indices = torch.topk(probabilities, 5) # need to get probabilities and the indexs
print(lm_prediction_scores)



print("\nTop-5 predictions for the next token:")
"""
YOUR CODE HERE
"""
for i in range(top5_probs.size(0)):
  token = tokenizer.decode(top5_indices[i])
  probability = top5_probs[i].item()
  print(f"Token: '{token}', Probability: {probability:.4f}")



{'input_ids': tensor([[15496,   616,  1438]]), 'attention_mask': tensor([[1, 1, 1]])}
CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[-35.2362, -35.3265, -38.9753,  ..., -44.4645, -43.9974, -36.4580],
         [-74.1302, -75.3139, -79.5631,  ..., -81.1064, -82.6454, -78.4387],
         [-54.6547, -56.0921, -62.8932,  ..., -66.3084, -68.8439, -60.8408]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=DynamicCache(layers=[DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer]), hidden_states=None, attentions=None, cross_attentions=None)
Logits shape : torch.Size([1, 3, 50257])
tensor([[[-35.2362, -35.3265, -38.9753,  ..., -44.4645, -43.9974, -36.4580],
         [-74.1302, -75.3139, -79.5631,  ..., -81.1064, -82.6454, -78.4387],
         [-54.6547, -56.0921, -62.8932,  ..., -66.3084, -68.8439, -60.8408]]],
       grad_fn=<UnsafeViewBackward0>)

Top-5 pred

### 2.5 - Key Takeaway

A language model is nothing mystical: it’s a *huge composition* of small, understandable layers trained to predict the next token in a sequence of tokens.

# 3 - Generation
Once an LLM is trained to predict the probabilities, we can generate text from the model. This process is called decoding or sampling.

At each step, the LLM outputs a **probability distribution** over the next token. It is the job of the decoding algorithm to pick the next token, and move on to the next token. There are different decoding algorithms and hyper-parameters to control the generaiton:
* **Greedy** → pick the single highest‑probability token each step (safe but repetitive).  
* **Top‑k / Nucleus (top‑p)** → sample from a subset of likely tokens (adds variety).
* **beam** -> applies beam search to pick tokens
* **Temperature** → a *creativity* knob. Higher values flatten the probability distribution.

### 3.1 - Greedy decoding

In [37]:
from transformers import AutoTokenizer, AutoModelForCausalLM
MODELS = {
    "gpt2": "gpt2",
}
tokenizers, models = {}, {}
# Load models and tokenizers
"""
YOUR CODE HERE
"""
tokenizers["gpt2"] = AutoTokenizer.from_pretrained("openai-community/gpt2")
models["gpt2"] = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")


def generate(model_key, prompt, strategy="greedy", max_new_tokens=100):
    tok, mdl = tokenizers[model_key], models[model_key]

    # Return the generations based on the provided strategy: greedy, top_k, top_p
    """
    YOUR CODE HERE
    """
    tokens = tok(prompt, return_tensors="pt")
    if strategy == "greedy": #lmao so bad
        output_tokens = mdl.generate(**tokens, max_new_tokens=max_new_tokens, do_sample=False)
    elif strategy == "top_k": #actually not as bad as I thought
        output_tokens = mdl.generate(**tokens, max_new_tokens=max_new_tokens, do_sample=True, top_k=80) # k is the threshold of probabilities
    elif strategy == "top_p": #honestly somehow the funniest results, but a p=0.9. Higher liklihood of getting stuck at lower p.
        output_tokens = mdl.generate(**tokens, max_new_tokens=max_new_tokens, do_sample=True, top_p=2.0) # p is equivalent to temperature
    else:
        return

    generated_text = tok.decode(output_tokens[0], skip_special_tokens=True)
    return generated_text

In [38]:
tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Greedy ==")
    print(generate("gpt2", prompt, "greedy", 80))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



== GPT-2 | Greedy ==


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and

== GPT-2 | Greedy ==


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is 2+2?

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

== GPT-2 | Greedy ==
Suggest a party theme.

The party theme is a simple, simple, and fun way to get your friends to join you.

The party theme is a simple, simple, and fun way to get your friends to join you. The party theme is a simple, simple, and fun way to get your friends to join you. The party theme is a simple, simple, and fun way to get your friends



Naively picking the single best token every time has the following issues in practice:

* **Loop**: “The cat is is is…”  
* **Miss long-term payoff**: the highest-probability word *now* might paint you into a boring corner later.

### 3.2 - Top-k or top-p sampling

In [35]:

tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Top-p ==")
    print(generate("gpt2", prompt, "top_p", 40))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



== GPT-2 | Top-p ==


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time the world had begun to grow dim and dark, and its inhabitants had a growing need for more intelligent devices to guide them, perhaps better suited to a better world. This was when the greatest breakthrough was

== GPT-2 | Top-p ==


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is 2+2?

Why? Because this is how the human body and mind processes information.

Now here's what you should understand: The 4-5 is NOT a number – it's a "positive

== GPT-2 | Top-p ==
Suggest a party theme.

You can create this party using a theme, such as "Avenue For Men". You can use an existing party theme if you want, or you can add theme by hand. The theme


### 3.3 - Try It Yourself

1. Scroll to the list called `tests`.
2. Swap in your own prompts or tweak the decoding strategy.  
3. Re‑run the cell and compare the vibes.

> **Tip:** Try the same prompt with `greedy` vs. `top_p` (0.9) and see how the tone changes. Notice especially how small temperature tweaks can soften or sharpen the prose.

* `strategy`: `"greedy"`, `"beam"`, `"top_k"`, `"top_p"`  
* `temperature`: `0.2 – 2.0`  
* `k` or `p` thresholds



# 4 - Completion vs. Instruction-tuned LLMs

We have seen that we can use GPT2 model to pass an input text and generate a new text. However, this model only continues the provided text. It is not engaging in a dialouge-like conversation and cannot be helpful by answering instructions. On the other hand, **instruction-tuned LLMs** like `Qwen-Chat` go through an extra training stage called **post-training** after the base “completion” model is finished. Because of post-training step, an instruction-tuned LLM will:

* **Read the entire prompt as a request,** not just as text to mimic.  
* **Stay in dialogue mode**. Answer questions, follow steps, ask clarifying queries.  
* **Refuse or safe-complete** when instructions are unsafe or disallowed.  
* **Adopt a consistent persona** (e.g., “Assistant”) rather than drifting into story continuation.


### 4.1 - Qwen1.5-8B vs. GPT2

In the code below we’ll feed the same prompt to:

* **GPT-2 (completion-only)** – it will simply keep writing in the same style.  
* **Qwen-Chat (instruction-tuned)** – it will obey the instruction and respond directly.

Comparing the two outputs makes the difference easy to see.

In [39]:
from transformers import AutoTokenizer, AutoModelForCausalLM
MODELS = {
    "gpt2": "gpt2",
    "qwen": "Qwen/Qwen1.5-1.8B-Chat"
}

"""
YOUR CODE HERE
"""
tokenizers, models = {}, {}
# Load models and tokenizers
tokenizers["gpt2"] = AutoTokenizer.from_pretrained(MODELS["gpt2"])
tokenizers["qwen"] = AutoTokenizer.from_pretrained(MODELS["qwen"])

models["gpt2"] = AutoModelForCausalLM.from_pretrained(MODELS["gpt2"])
models["qwen"] = AutoModelForCausalLM.from_pretrained(MODELS["qwen"])


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

'\nYOUR CODE HERE\n'


We downloaded two tiny checkpoints: `GPT‑2` (124 M parameters) and `Qwen‑1.5‑Chat` (1.8 B). If the cell took a while, that was mostly network time. Models are stored locally after the first run.

Let's now generate text and compare two models.


In [40]:

tests=[("Once upon a time","greedy"),("What is 2+2?","top_k"),("Suggest a party theme.","top_p")]
for prompt,strategy in tests:
    for key in ["gpt2","qwen"]:
        print(f"\n== {key.upper()} | {strategy} ==")
        print(generate(key,prompt,strategy,80))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



== GPT2 | greedy ==


The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and

== QWEN | greedy ==


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, there was a young girl named Lily who lived in a small village nestled among the rolling hills of her home. Lily had always been fascinated by nature and spent most of her days exploring the surrounding forests, streams, and meadows. She loved to sing songs about the birds that sang in the trees, the flowers that bloomed in the fields, and the animals that roamed free.
One day

== GPT2 | top_k ==
What is 2+2? Can I add 2 if I don't want 2?

Any questions/questions? Please address them to my@uwe.edu. The answer could be simply mentioned as the above issue.

Thank you!

== QWEN | top_k ==


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is 2+2? 2 + 2 equals 4. The answer is 4.

== GPT2 | top_p ==
Suggest a party theme. To play a different theme, add an external icon to your app (e.g., a custom slider). Once you've created a theme, simply edit it and click Insert.

App Theme Example

Creating a themed layout can be accomplished using many different approaches. If you want to have a custom layout, you can use the theme in your app by adding the theme to your library,

== QWEN | top_p ==
Suggest a party theme. Great idea! Here's a fun and energetic party theme that would be perfect for hosting a summer bash:

Name: Tropical Paradise Party

Party Details:
- Time: Saturday afternoon, June 15th
- Location: A large rooftop in the city center, with a panoramic view of the local park and city skyline.
- Dress Code: Casual beach-inspired attire for both adults and kids (w


# 5. (Optional) A Small LLM Playground

### 5.1 ‑ Interactive Playground

Enter a prompt, pick a model and decoding strategy, adjust the temperature, and press **Generate** to watch the model respond.


In [63]:
import ipywidgets as widgets
from IPython.display import display, Markdown
from transformers import AutoTokenizer, AutoModelForCausalLM

#list of available models
AVAILABLE_MODELS = {
    "gpt2": "gpt2",
    "qwen": "Qwen/Qwen1.5-1.8B-Chat"
}
AVAILABLE_STRATEGIES = {
    "greedy":"greedy",
    "top_k":"top_k",
    "top_p":"top_p"
}

tokenizers, models = {}, {}
# Load models and tokenizers


# Make sure models and tokenizers are loaded
try:
    tokenizers
    models
except NameError:
    raise RuntimeError("Please run the earlier setup cells that load the models before using the playground.")

def generate_playground(model_key, prompt, strategy="greedy", temperature=1.0, max_new_tokens=100, top_k_input=80):
    # Generation code
    if model_key not in AVAILABLE_MODELS:
      print("Invalid Model. Please select from the following list")
      for i in AVAILABLE_MODELS:
        print(i)
      return
    else:
      print("Selected Model:", model_key)


    if strategy not in AVAILABLE_STRATEGIES:
      print("Invalid Strategy. Please select from the following list")
      for i in AVAILABLE_STRATEGIES:
        print(i)
      return
    else:
      print("Selected Strategy:", strategy)


    tokenizers[model_key] = AutoTokenizer.from_pretrained(AVAILABLE_MODELS[model_key])
    models[model_key] = AutoModelForCausalLM.from_pretrained(AVAILABLE_MODELS[model_key])

    tokens = tokenizers[model_key](prompt, return_tensors="pt")
    print("Info on tokenization", tokens)
    if strategy == "greedy":
        output_tokens = models[model_key].generate(**tokens, max_new_tokens=max_new_tokens, do_sample=False)
    elif strategy == "top_k":
        output_tokens = models[model_key].generate(**tokens, max_new_tokens=max_new_tokens, do_sample=True, top_k=top_k_input)
    elif strategy == "top_p":
        output_tokens = models[model_key].generate(**tokens, max_new_tokens=max_new_tokens, do_sample=True, top_p=temperature)
    else:
        return

    print("Output Token shape", output_tokens.shape)
    generated_text = tokenizers[model_key].decode(output_tokens[0], skip_special_tokens=True)
    return generated_text



# Your code to build boxes, dropdowns, and other elements in the UI using widgets and creating the UI using widgets.vbox and display.
# Refer to https://ipywidgets.readthedocs.io/en/stable/
"""
YOUR CODE HERE
"""
output = generate_playground("gpt2", "Hello there","top_p", temperature=0.2, max_new_tokens=50)
print(output)



Selected Model: gpt2
Selected Strategy: top_p


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Info on tokenization {'input_ids': tensor([[15496,   612]]), 'attention_mask': tensor([[1, 1]])}
Output Token shape torch.Size([1, 52])
Hello there, I'm back. I'm back. I'm back. I'm back. I'm back. I'm back. I'm back. I'm back. I'm back. I'm back. I'm back. I'm back. I



## 🎉 Congratulations!

You’ve just learned, explored, and inspected a real **LLM**. In one project you:
* Learned how **tokenization** works in practice
* Used `tiktoken` library to load and experiment with most advanced tokenizers.
* Explored LLM architecture and inspected GPT2 blocks and layers
* Learned decoding strategies and used `top-p` to generate text from GPT2
* Loaded a powerful chat model, `Qwen1.5-8B` and generated text
* Built an LLM playground


👏 **Great job!** Take a moment to celebrate. You now have a working mental model of how LLMs work. The skills you used here power most LLMs you see everywhere.
